In [13]:
import pandas as pd
"""
# Charger le fichier original
df = pd.read_csv('data/EDdA_dataframe_withContent.tsv', sep='\t')

# Nettoyage des colonnes non nécessaires
columns_to_drop = ['firstParagraph', 'contentWithoutClass', 'ensemble_domaine_enccre', 'domaine_enccre', 'normClass']
df_cleaned = df.drop(columns=columns_to_drop)

# Enlever les trois premières lignes
df_cleaned = df_cleaned.iloc[3:]

# Sauvegarder les données nettoyées
df_cleaned.to_csv('data/EDdA_dataframe_cleaned.tsv', sep='\t', index=False)
"""
# charger le fichier nettoyé
df_cleaned = pd.read_csv('data/EDdA_dataframe_cleaned.tsv', sep='\t')
df_cleaned

,volume,numero,head,classEDdA,author,id_enccre,content,nb_words
0,1,5,"A, a & a","ordre Encyclopéd. Entend. Science de l'homme, ...",Dumarsais5,v1-1-0,"A, a & a s.m. (ordre Encyclopéd.\nEntend. Scie...",1092
1,1,6,A,unclassified,Dumarsais5,v1-1-1,"A, mot, est 1. la troisieme personne du présen...",381
2,1,7,A,unclassified,Dumarsais,v1-1-2,"A, préposition vient du latin à, à dextris, à ...",3067
3,1,8,A,unclassified,Mallet,v1-1-3,"A, étoit une lettre numérale parmi les Anciens...",285
4,1,9,"A, lettre symbolique",unclassified,Mallet,v1-1-4,"A, lettre symbolique, étoit un hiéroglyphe che...",118
...,...,...,...,...,...,...,...,...
74182,17,3204,TRANSFUGE,Art. milit.,Saint-Lambert5,v17-2197-0,"TRANSFUGE, s. m. (Art. milit.) La plus grande\...",9576
74183,17,3205,VÉNUS,Astronom.,unsigned,v17-2198-0,"VÉNUS, (Astronom.) satellites de Vénus. Depuis...",3754
74184,17,3206,VERS falisque,Poésie latine.,Jaucourt,v17-2199-0,"VERS falisque, (Poésie latine.) vers latin de\...",97
74185,17,3207,"VIBRATION, ou OSCILLATION",Horlog.,Romilly5,v17-2200-0,"VIBRATION, ou OSCILLATION, s. f. (Horlog.) ter...",4945


In [1]:
# Function to generate the 'id_enccre' based on the observed pattern
def generate_id_enccre(row, index):
    return f"v{int(row['volume'])}-{int(row['numero'])}-{index}"

# Apply the function to fill in the missing 'id_enccre' values
df_cleaned['id_enccre'] = df_cleaned.apply(lambda row: row['id_enccre'] if pd.notna(row['id_enccre']) else generate_id_enccre(row, row.name), axis=1)

# Verify that there are no more missing 'id_enccre' values
missing_id_enccre_after = df_cleaned[df_cleaned['id_enccre'].isna()]
missing_id_enccre_after

NameError: name 'df_cleaned' is not defined

In [ ]:
# Export en corrected
df_cleaned.to_csv('data/EDdA_dataframe_cleaned_corrected.tsv', sep='\t', index=False)

In [12]:
# Faire un sample de 100 lignes
df_sample = df_cleaned.sample(3333)
df_sample.to_csv('data/EDdA_dataframe_sample.tsv', sep='\t', index=False)

## Compte du nombre de tokens

In [16]:
import pandas as pd
import tiktoken

# Charger le fichier nettoyé
df_cleaned = pd.read_csv('data/EDdA_dataframe_cleaned_corrected.tsv', sep='\t')

# Instancier le tokenizer pour l'encodage des tokens
tokenizer = tiktoken.get_encoding("cl100k_base")

# Convertir la colonne 'content' en chaîne de caractères pour éviter les erreurs
df_cleaned['content'] = df_cleaned['content'].astype(str)

# Calculer le nombre total de tokens pour chaque texte de la colonne 'content'
df_cleaned['token_count'] = df_cleaned['content'].apply(lambda x: len(tokenizer.encode(x)))

# Calculer le nombre total de tokens dans le fichier
total_tokens = df_cleaned['token_count'].sum()

print(f"Nombre total de tokens : {total_tokens}")

Nombre total de tokens : 36356811


## Test de quantization

In [3]:
# Modèle text-embedding-3-small
cost_small = (total_tokens / 1_000_000) * 0.020
batch_cost_small = (total_tokens / 1_000_000) * 0.010

# Modèle text-embedding-3-large
cost_large = (total_tokens / 1_000_000) * 0.130
batch_cost_large = (total_tokens / 1_000_000) * 0.065

# Modèle ada v2
cost_ada = (total_tokens / 1_000_000) * 0.100
batch_cost_ada = (total_tokens / 1_000_000) * 0.050

cost_small, batch_cost_small, cost_large, batch_cost_large, cost_ada, batch_cost_ada

C:\Users\mogui\AppData\Local\Temp\ipykernel_28732\3865763844.py:18: RuntimeWarning: invalid value encountered in cast
  int8_embeddings = np.clip(int8_embeddings, -128, 127).astype(np.int8).tolist()


Fichier JSON-LD avec embeddings quantifiés sauvegardé dans : data/EDdA_knowledge_graph_int8.jsonld


In [3]:
# On prend le fichier EDdA_knowledge_graph_sample.jsonld pour le transformer en json
import json

# Charger le fichier jsonld
with open('data/EDdA_knowledge_graph_sample.jsonld', 'r', encoding='utf-8') as f: 
    data = json.load(f)

# Sauvegarder le fichier en json
with open('data/EDdA_knowledge_graph_sample.json', 'w') as f:
    json.dump(data, f, indent=4)